# Day 2b - Agent 工具最佳实践（LangChain可本地运行版）

🚀 **Agent 工具模式与最佳实践**

欢迎来到 5 天 Agent 课程的第二天！

在本笔记本中，我们将实现与 **真实 MCP 服务器** 的连接，并处理具有 **人机交互 (Human-in-the-Loop)** 的长时间运行操作。

**原版**: Google ADK Team  
**改写**: LangChain 实现 (集成官方 MCP SDK)

在本笔记本中，你将学习如何：

- ✅ 使用 `mcp` Python SDK 连接到真实的 MCP 服务器 (如 `@modelcontextprotocol/server-everything`)
- ✅ 将 MCP 工具桥接到 LangChain Agent
- ✅ 实现 **人机交互 (Human-in-the-Loop)** 审批工作流
---
**原版**: Google ADK Team (Kaggle 5-Day Agent Course)  
**改写**: LangChain 实现
> **ℹ️ 声明**: 本教程改编自 Google [5-Day AI Agents Course](https://www.kaggle.com/learn-guide/5-day-agents)，由 LangChain 重写以适配本地运行。内容仅供学习研究。


## ⚙️ 第 1 部分：设置 (Setup)

### 1.1 切换内核
- 点击右上角的 Kernel (内核) 选择区域。
- 在列表中选择 Python (5-DAY-AGENTS)。

> 关于如何初始化环境和创建内核，请查看 [README 如何运行](../README.md#如何运行)

### 1.2 安装依赖
除了 LangChain，我们还需要安装 `mcp` 官方 SDK。

In [ ]:
# 优选 uv 包管理器
!uv pip install langchain langchain-openai langchain-community python-dotenv mcp nest_asyncio

### 1.3 导入依赖库

In [ ]:
import os
import json
import getpass
import asyncio
import nest_asyncio
from dotenv import load_dotenv

print("✅ 依赖库加载完成")

### 1.4 配置 API Key

In [ ]:
# 允许在 Jupyter 中嵌套运行 asyncio
nest_asyncio.apply()

# 加载环境变量
load_dotenv()

def configure_api_keys():
    """加载并验证 API 配置"""
    model_name = os.environ.get("MODEL_NAME")
    api_key = os.environ.get("OPENAI_API_KEY")
    base_url = os.environ.get("OPENAI_API_BASE")

    if not api_key:
        print("❌ 错误: 未在 .env 文件中找到 OPENAI_API_KEY")
        return None
    
    # 打印部分 Key 以验证 (隐藏敏感信息)
    masked_key = f"{api_key[:8]}..." if api_key else "None"
    print(f"✅ API Key 已加载: {masked_key}")
    print(f"✅ 使用模型: {model_name}")
    if base_url:
        print(f"✅ API Base URL: {base_url}")
    
    return {
        "model_name": model_name,
        "api_key": api_key,
        "base_url": base_url,
        "extra_body": json.loads(os.environ.get("EXTRA_BODY", "{}"))
    }

config = configure_api_keys()

## 🧰 第 2 部分：连接真实 MCP 服务器

我们将使用 `npx` 运行 `@modelcontextprotocol/server-everything`，并通过 `mcp` Python SDK与之通信。

为了让 LangChain Agent 能使用这些工具，我们需要编写一个简单的适配器，将 MCP 工具转换为 LangChain 工具。


In [ ]:
from mcp import StdioServerParameters, ClientSession
from mcp.client.stdio import stdio_client
from langchain_core.tools import StructuredTool
from pydantic import BaseModel, create_model
import inspect

class MCPClientService:
    def __init__(self, command, args):
        self.server_params = StdioServerParameters(command=command, args=args)
        self.session = None
        self.exit_stack = None
        self.read = None
        self.write = None
    
    async def __aenter__(self):
        # 建立连接
        self.client_ctx = stdio_client(self.server_params)
        self.read, self.write = await self.client_ctx.__aenter__()
        
        self.session_ctx = ClientSession(self.read, self.write)
        self.session = await self.session_ctx.__aenter__()
        
        await self.session.initialize()
        return self
        
    async def __aexit__(self, exc_type, exc_val, exc_tb):
        if self.session_ctx:
            await self.session_ctx.__aexit__(exc_type, exc_val, exc_tb)
        if self.client_ctx:
            await self.client_ctx.__aexit__(exc_type, exc_val, exc_tb)

    async def get_langchain_tools(self, tool_filter=None):
        """获取 MCP 工具并转换为 LangChain 工具"""
        if not self.session:
            raise RuntimeError("Session not initialized")
            
        mcp_tools_list = await self.session.list_tools()
        langchain_tools = []
        
        for mcp_tool in mcp_tools_list.tools:
            if tool_filter and mcp_tool.name not in tool_filter:
                continue
                
            # print(f"  - 发现工具: {mcp_tool.name}")
            
            # 动态创建参数 Schema
            fields = {}
            if mcp_tool.inputSchema and "properties" in mcp_tool.inputSchema:
                for name, schema in mcp_tool.inputSchema["properties"].items():
                    # 简化处理：默认视为字符串，实际应根据 type 映射
                    fields[name] = (str, ... if name in mcp_tool.inputSchema.get("required", []) else None)
            
            args_schema = create_model(f"{mcp_tool.name}Schema", **fields)
            
            # 创建包装函数
            # 创建包装函数
            # fix: 使用工厂函数避免闭包变量捕获问题
            def make_tool_func(tool_name):
                async def _tool_func(**kwargs):
                    # print(f"    [MCP调用] {tool_name} with {kwargs}")
                    result = await self.session.call_tool(tool_name, arguments=kwargs)
                    
                    # 处理文本和图像结果
                    output = []
                    for content in result.content:
                        if content.type == 'text':
                            output.append(content.text)
                        elif content.type == 'image':
                            output.append(f"[Image: {content.mimeType}, data_len={len(content.data)}]")
                            
                            # 在 Notebook 环境下直接显示图片
                            try:
                                from IPython.display import display, Image
                                import base64
                                display(Image(data=base64.b64decode(content.data)))
                            except ImportError:
                                pass # 非 Notebook 环境忽略
                                
                    return "\n".join(output)
                return _tool_func

            _tool_func = make_tool_func(mcp_tool.name)

            langchain_tools.append(StructuredTool.from_function(
                func=None,
                coroutine=_tool_func, # LangChain 支持 async 工具
                name=mcp_tool.name,
                description=mcp_tool.description,
                args_schema=args_schema
            ))
            
        return langchain_tools

print("✅ MCPClientService 类已定义")

### 2.2 连接并获取工具
现在我们启动服务器并获取 `getTinyImage` 工具。

In [ ]:
# 注意：这需要系统中安装了 npm/npx
mcp_service = MCPClientService(
    command="npx", 
    args=["-y", "@modelcontextprotocol/server-everything"]
)

# 我们将在一个 async 上下文中运行 Agent
async def setup_agent():
    print("🔌 正在连接 MCP 服务器...")
    async with mcp_service as service:
        print("✅ 已连接。正在获取工具...")
        
        # 获取工具 (只筛选 getTinyImage)
        tools = await service.get_langchain_tools(tool_filter=["getTinyImage"])
        print(f"✅ 成功加载 {len(tools)} 个工具: {[t.name for t in tools]}")
        
        return tools

# 由于 Notebook 的生命周期，我们这里简单演示从连接中获取工具对象
# 实际 Agent 运行时需要保持连接开启。我们将把 Agent 运行也放在 async 函数中。


### 2.3 运行使用 MCP 工具的 Agent

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

async def run_image_agent_demo():
    async with mcp_service as service:
        # 1. 获取工具
        tools = await service.get_langchain_tools(tool_filter=["getTinyImage"])
        
        # 2. 初始化 Agent
        llm = ChatOpenAI(model=config["model_name"], temperature=0.7)
        prompt = ChatPromptTemplate.from_messages([
            ("system", "你是一个助手。请使用工具生成图片。"),
            ("human", "{input}"),
            ("placeholder", "{agent_scratchpad}"),
        ])
        
        agent = create_tool_calling_agent(llm, tools, prompt)
        agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
        
        # 3. 运行
        print("\n🤖 User: 生成一张示例小图")
        response = await agent_executor.ainvoke({"input": "生成一张示例小图"})
        print(f"🤖 Agent: {response['output']}")

# 运行演示
await run_image_agent_demo()

## 🔄 第 3 部分：长时间运行的操作 (人机交互)

这部分内容与之前相同，我们使用 LangChain 本地工具模拟审批流。

注意：由于我们现在在 async 环境中运行（为了支持 MCP），我们的工具也可以是 async 的，或者直接使用 input() 阻塞（在本地 Demo 中通常没问题）。


In [ ]:
from langchain_core.tools import tool

@tool
def place_shipping_order(num_containers: int, destination: str) -> str:
    """
    处理发货订单。如果集装箱数量 > 5，将触发人工审批流程。
    """
    LARGE_ORDER_THRESHOLD = 5
    
    print(f"    🚢 [工具] 正在处理订单: {num_containers} 个集装箱 -> {destination}...")
    
    if num_containers <= LARGE_ORDER_THRESHOLD:
        order_id = f"ORD-{num_containers}-AUTO"
        return f"✅ 自动批准: {num_containers} 个集装箱已发往 {destination}。订单号: {order_id}"
    
    print(f"\n    ⚠️  检测到大额订单 > {LARGE_ORDER_THRESHOLD}")
    print(f"    ⏸️  [系统暂停] 等待人工批准...")
    
    # 阻塞式输入
    user_decision = input(f"    🤔 批准运送 {num_containers} 个集装箱到 {destination} 吗？(输入 yes/no): ").strip().lower()
    
    if user_decision in ['yes', 'y', 'ok', 'approve', '是', '好']:
        order_id = f"ORD-{num_containers}-HUMAN"
        print(f"    ✅ 人工已批准。")
        return f"✅ 人工已批准: {num_containers} 个集装箱已发往 {destination}。订单号: {order_id}"
    else:
        print(f"    ❌ 人工已拒绝。")
        return f"❌ 人工已拒绝: 订单已取消。"

print("✅ 工具 `place_shipping_order` 已就绪。")

In [ ]:
# 创建物流 Agent
shipping_tools = [place_shipping_order]

shipping_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一名物流协调员。使用工具处理订单。"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

llm = ChatOpenAI(model=config["model_name"], temperature=0)
shipping_agent = create_tool_calling_agent(llm, shipping_tools, shipping_prompt)
shipping_runner = AgentExecutor(agent=shipping_agent, tools=shipping_tools, verbose=True, handle_parsing_errors=True)

# 演示
print("--- 演示: 大额订单 ---")
print("(系统将提示你输入...)")
await shipping_runner.ainvoke({"input": "运送 3 个集装箱到新加坡"})
await shipping_runner.ainvoke({"input": "运送 10 个集装箱到鹿特丹"})

## 📊 总结

我们成功地：
1. **连接了真实的 MCP 服务器**: 使用 `npx` 启动并通过 `mcp` SDK 通信。
2. **集成了 LangChain**: 编写了适配器将 MCP 工具动态转换为 LangChain 工具。
3. **保持了交互性**: 实现了 Human-in-the-Loop 用于敏感操作。
